# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-07 00:00:00  to  2020-04-08 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,119,124,5,4.201681
1,02,255,260,5,1.960784
2,03,52,53,1,1.923077
3,04,29,30,1,3.448276
4,05,64,61,-3,-4.687500


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,3169,3154,-15,-0.473335
94,94,1956,2046,90,4.601227
92,92,1932,1987,55,2.846791
93,93,1461,1499,38,2.600958
69,69,1174,1150,-24,-2.044293


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -28  Mean:  3.6041666666666665  Max:  167


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -40.0  Mean %:  0.9754651525631078  Max %:  115.38461538461537


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
4,05,64,61,-3,-4.687500
7,08,80,75,-5,-6.250000
9,10,157,153,-4,-2.547771
10,11,81,73,-8,-9.876543
11,12,55,52,-3,-5.454545
15,16,12,11,-1,-8.333333
17,18,54,51,-3,-5.555556
20,22,69,66,-3,-4.347826
22,24,26,23,-3,-11.538462
25,27,66,50,-16,-24.242424


In [11]:
print(stats.query("diff < 0").shape)

(42, 5)


#### <br> Departments with highest evolution of hospitalizations (total number)

In [12]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
91,91,958,1125,167,17.432150
94,94,1956,2046,90,4.601227
92,92,1932,1987,55,2.846791
78,78,992,1040,48,4.838710
93,93,1461,1499,38,2.600958


#### <br> Departments with highest rate of hospitalizations (%)

In [13]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
21,23,13,28,15,115.384615
91,91,958,1125,167,17.432150
40,40,26,29,3,11.538462
32,32,26,29,3,11.538462
82,82,18,20,2,11.111111


#### <br> Focus on some departments

In [14]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,3169,3154,-15,-0.473335


In [15]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,191,191,0,0.0


In [16]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,230,229,-1,-0.434783
